<a href="https://colab.research.google.com/github/Vadim-sys-create/TTS/blob/main/%D0%9A%D0%BE%D0%BF%D0%B8%D1%8F_%D0%B1%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82%D0%B0_%22_untitled19_ipynb_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gtts

In [ ]:
!pip install --force-reinstall gtts

  Using cached gTTS-2.5.4-py3-none-any.whl.metadata (4.1 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached click-8.1.8-py3-none-any.whl.metadata (2.3 kB)
  Using cached charset_normalizer-3.4.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (35 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached urllib3-2.3.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached certifi-2025.1.31-py3-none-any.whl.metadata (2.5 kB)
Using cached gTTS-2.5.4-py3-none-any.whl (29 kB)
Using cached click-8.1.8-py3-none-any.whl (98 kB)
Using cached requests-2.32.3-py3-none-any.whl (64 kB)
Using cached certifi-2025.1.31-py3-none-any.whl (166 kB)
Using cached charset_normalizer-3.4.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (143 kB)
Using cached idna-3.10-py3-none-any.whl (70 kB)
Using cached urllib3-2.3.0-py3-none-any.whl (128 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.3.0
    Unin

In [ ]:
!pip install flask

In [ ]:
!python app.py

2025-03-02 10:33:38.475346: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1740911618.624095    2442 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1740911618.647402    2442 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-02 10:33:38.722348: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
Traceback (most recent call last):
  File "/content/app.py", line 4, in <module>
    from fuzzywuzzy import process
ModuleNot

In [ ]:
!pip install aiogram transformers gtts fuzzywuzzy langdetect nest_asyncio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 13.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 612.8/612.8 kB 30.1 MB/s eta 0:00:00
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993222 sha256=f146ba80352102d4c8a413edc047c549ba7549f63802962414bed110e02e8612
  Stored in directory: /root/.cache/pip/wheels/0a/f2/b2/e5ca405801e05eb7c8ed5b3b4bcf1fcabcd6272c167640072e
Successfully built langdetect


In [ ]:
# Установка библиотек
!pip install aiogram transformers gtts fuzzywuzzy langdetect nest_asyncio

# Импорт библиотек
import os
from aiogram import Bot, Dispatcher, types
from aiogram.filters import Command
from aiogram.enums import ParseMode
from aiogram.client.default import DefaultBotProperties
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from gtts import gTTS
from fuzzywuzzy import process
from langdetect import detect
import nest_asyncio

# Применяем nest_asyncio для работы в Colab
nest_asyncio.apply()

# Загрузка модели и токенизатора
model_name = "sberbank-ai/rugpt3large_based_on_gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Ваши ответы на вопросы
my_responses = {
    "Как тебя зовут?": "Меня зовут Аватар, и я здесь, чтобы помочь тебе!",
    "What is your name?": "My name is Avatar, and I'm here to help you!",
    "¿Cómo te llamas?": "Me llamo Avatar, y estoy aquí para ayudarte.",
    "Comment tu t'appelles?": "Je m'appelle Avatar, et je suis là pour vous aider.",
}

# Функция для генерации ответа
def generate_response(question, max_length=50):
    # Поиск наиболее похожего вопроса в словаре
    match, score = process.extractOne(question, my_responses.keys())

    # Если совпадение достаточно точное (например, score > 80), возвращаем ответ
    if score > 80:
        return my_responses[match]

    # Если совпадение неточное, генерируем ответ с помощью модели
    inputs = tokenizer(question, return_tensors="pt")
    outputs = model.generate(
        inputs["input_ids"],
        max_length=max_length,
        num_return_sequences=1,
        no_repeat_ngram_size=2,
        top_k=50,
        top_p=0.95,
        temperature=0.7,
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Функция для определения языка текста
def detect_language(text):
    try:
        return detect(text)
    except:
        return 'ru'  # По умолчанию используем русский язык

# Функция для создания аудио из текста
def create_audio_from_text(text, output_file="response.mp3"):
    lang = detect_language(text)  # Автоматически определяем язык
    tts = gTTS(text, lang=lang)
    tts.save(output_file)
    return output_file

# Инициализация бота
API_TOKEN = '7954916275:AAEoTnpSPG_DRajIGC859AqOmVejyrn7VL0'  # Замените на ваш токен
bot = Bot(token=API_TOKEN, default=DefaultBotProperties(parse_mode=ParseMode.HTML))
dp = Dispatcher()

# Гифки и картинки для ответов
happy_gif = "https://media.giphy.com/media/3o7aTskHEUdgCQAXde/giphy.gif"  # Пример гифки
thinking_image = "https://example.com/thinking.jpg"  # Пример картинки

# Обработчик команды /start
@dp.message(Command("start"))
async def send_welcome(message: types.Message):
    await message.answer("Привет! Я Аватар. Задай мне вопрос!")
    await message.answer_animation(happy_gif)  # Отправляем гифку

# Обработчик текстовых сообщений
@dp.message()
async def echo(message: types.Message):
    user_input = message.text

    if user_input.lower() in ["выход", "exit", "quit"]:
        await message.answer("Аватар: Пока!")
        return

    try:
        # Генерация ответа
        response = generate_response(user_input)
        await message.answer(f"Аватар: {response}")

        # Создание аудио
        audio_file = create_audio_from_text(response)

        # Проверка, создан ли аудиофайл
        if os.path.exists(audio_file):
            print("Аудиофайл создан.")  # Для отладки

            # Отправка аудио (используем путь к файлу)
            with open(audio_file, "rb") as audio:
                await message.answer_audio(audio)
        else:
            print("Ошибка: аудиофайл не создан.")  # Для отладки
            await message.answer("Не удалось создать аудио.")

        # Отправка гифки или картинки
        if "радост" in response.lower():
            await message.answer_animation(happy_gif)  # Отправляем гифку
        else:
            await message.answer_photo(thinking_image)  # Отправляем картинку

        # Удаление временного файла
        if os.path.exists(audio_file):
            os.remove(audio_file)
    except Exception as e:
        await message.answer(f"Произошла ошибка: {str(e)}")  # Исправлено: str(e) для корректного вывода ошибки

# Запуск бота
async def start_bot():
    await dp.start_polling(bot)

# Запуск бота в Colab
await start_bot()

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
ERROR:aiogram.event:Cause exception while process update id=995587322 by bot id=7954916275
TelegramBadRequest: Telegram server says - Bad Request: can't parse entities: Unsupported start tag "_io.bufferedreader" at byte offset 173
Traceback (most recent call last):
  File "<ipython-input-19-d12a11f58cce>", line 106, in echo
    await message.answer_audio(audio)
          ^^^^^^^^^^^^^^^^^^^^^

Аудиофайл создан.


In [ ]:
!pip install datasets

In [ ]:
# Установка необходимых библиотек
!pip install datasets transformers

# Импорт библиотек
from transformers import GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments
from datasets import load_dataset

# Загрузка модели и токенизатора
model_name = "sberbank-ai/rugpt3large_based_on_gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Загрузка данных для дообучения
dataset = load_dataset('text', data_files={'train': '/content/data.txt'})

# Токенизация данных
def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Настройка параметров обучения
training_args = TrainingArguments(
    output_dir="./results",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=2,
    save_steps=10_000,
    save_total_limit=2,
)

# Создание Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
)

# Дообучение модели
trainer.train()

# Сохранение модели
model.save_pretrained("./fine_tuned_model")
tokenizer.save_pretrained("./fine_tuned_model")

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: institut-rpik (institut-rpik-robbinsdale-area-schools) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [ ]:
async def handle_message(update: Update, context: ContextTypes.DEFAULT_TYPE):
    try:
        user_message = update.message.text
        bot_response = generate_text(user_message)  # Генерация ответа через Hugging Face
        await update.message.reply_text(bot_response)
    except Exception as e:
        await update.message.reply_text(f"Произошла ошибка: {str(e)}")

NameError: name 'Update' is not defined

In [ ]:
def generate_text(prompt):
    return generator(
        prompt,
        max_length=100,  # Максимальная длина ответа
        num_return_sequences=1,  # Количество вариантов ответа
        no_repeat_ngram_size=2,  # Избегание повторяющихся фраз
        top_k=50,  # Ограничение на выбор токенов
        top_p=0.95,  # Фильтрация по вероятности
        temperature=0.7  # Контроль случайности
    )[0]["generated_text"]

In [ ]:
import logging

# Настройка логирования
logging.basicConfig(
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s", level=logging.INFO
)
logger = logging.getLogger(__name__)

# Логирование в обработчиках
async def start(update: Update, context: ContextTypes.DEFAULT_TYPE):
    logger.info(f"Пользователь {update.message.from_user.username} запустил бота.")
    await update.message.reply_text("Привет! Я ваш Нейрокуратор с Hugging Face Transformers. Задайте свой вопрос.")

async def handle_message(update: Update, context: ContextTypes.DEFAULT_TYPE):
    try:
        user_message = update.message.text
        logger.info(f"Пользователь {update.message.from_user.username} написал: {user_message}")
        bot_response = generate_text(user_message)
        await update.message.reply_text(bot_response)
    except Exception as e:
        logger.error(f"Ошибка: {str(e)}")
        await update.message.reply_text(f"Произошла ошибка: {str(e)}")

NameError: name 'Update' is not defined

In [ ]:
async def help_command(update: Update, context: ContextTypes.DEFAULT_TYPE):
    help_text = """
    Я ваш Нейрокуратор! Вот что я умею:
    - Отвечать на ваши вопросы.
    - Генерировать текст по вашему запросу.
    Просто напишите мне что-нибудь, и я постараюсь помочь!
    """
    await update.message.reply_text(help_text)

# Регистрация обработчика команды /help
application.add_handler(CommandHandler("help", help_command))

NameError: name 'Update' is not defined

In [ ]:
# Установка модели GPT-Neo
!pip install transformers

# Инициализация модели GPT-Neo
generator = pipeline("text-generation", model="EleutherAI/gpt-neo-2.7B")

NameError: name 'pipeline' is not defined

In [ ]:
# Словарь для хранения контекста
user_context = {}

async def handle_message(update: Update, context: ContextTypes.DEFAULT_TYPE):
    try:
        user_id = update.message.from_user.id
        user_message = update.message.text

        # Добавляем новое сообщение в контекст
        if user_id not in user_context:
            user_context[user_id] = []
        user_context[user_id].append({"role": "user", "content": user_message})

        # Генерация ответа с учётом контекста
        bot_response = generate_text(user_context[user_id])
        user_context[user_id].append({"role": "assistant", "content": bot_response})

        await update.message.reply_text(bot_response)
    except Exception as e:
        logger.error(f"Ошибка: {str(e)}")
        await update.message.reply_text(f"Произошла ошибка: {str(e)}")

NameError: name 'Update' is not defined

In [ ]:
# Установка библиотек
!pip install python-telegram-bot transformers torch nest_asyncio

# Применение nest_asyncio для работы в Colab
import nest_asyncio
nest_asyncio.apply()

# Импорт библиотек
import torch
from transformers import pipeline
from telegram import Update  # Импортируем Update
from telegram.ext import Application, CommandHandler, MessageHandler, filters, ContextTypes
import logging

# Настройка логирования
logging.basicConfig(
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s", level=logging.INFO
)
logger = logging.getLogger(__name__)

# Токен вашего Telegram-бота
TELEGRAM_TOKEN = "7954916275:AAEoTnpSPG_DRajIGC859AqOmVejyrn7VL0"

# Инициализация модели Hugging Face Transformers
generator = pipeline("text-generation", model="gpt2")  # Используем модель GPT-2

# Функция для генерации текста
def generate_text(prompt):
    return generator(
        prompt,
        max_length=100,
        num_return_sequences=1,
        no_repeat_ngram_size=2,
        top_k=50,
        top_p=0.95,
        temperature=0.7
    )[0]["generated_text"]

# Обработчик команды /start
async def start(update: Update, context: ContextTypes.DEFAULT_TYPE):
    logger.info(f"Пользователь {update.message.from_user.username} запустил бота.")
    await update.message.reply_text("Привет! Я ваш Нейрокуратор! Задайте свой вопрос.")

# Обработчик команды /help
async def help_command(update: Update, context: ContextTypes.DEFAULT_TYPE):
    help_text = """
    Я ваш Нейрокуратор! Вот что я умею:
    - Отвечать на ваши вопросы.
    - Генерировать текст по вашему запросу.
    Просто напишите мне что-нибудь, и я постараюсь помочь!
    """
    await update.message.reply_text(help_text)

# Обработчик текстовых сообщений
async def handle_message(update: Update, context: ContextTypes.DEFAULT_TYPE):
    try:
        user_message = update.message.text
        logger.info(f"Пользователь {update.message.from_user.username} написал: {user_message}")
        bot_response = generate_text(user_message)
        await update.message.reply_text(bot_response)
    except Exception as e:
        logger.error(f"Ошибка: {str(e)}")
        await update.message.reply_text(f"Произошла ошибка: {str(e)}")

# Основная функция
def main():
    # Создание приложения бота
    application = Application.builder().token(TELEGRAM_TOKEN).build()

    # Регистрация обработчиков
    application.add_handler(CommandHandler("start", start))
    application.add_handler(CommandHandler("help", help_command))
    application.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, handle_message))

    # Запуск бота
    application.run_polling()

if __name__ == "__main__":
    main()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 676.1/676.1 kB 8.5 MB/s eta 0:00:00


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


RuntimeError: Cannot close a running event loop

In [ ]:
!pip install python-telegram-bot transformers torch nest_asyncio

In [ ]:
!pip list | grep -E "telegram|transformers|torch|nest_asyncio"

python-telegram-bot                21.11.1
sentence-transformers              3.3.1
torch                              2.5.1+cu121
torchaudio                         2.5.1+cu121
torchsummary                       1.5.1
torchvision                        0.20.1+cu121
transformers                       4.47.1


In [ ]:
async def handle_message(update: Update, context: ContextTypes.DEFAULT_TYPE):
    try:
        user_message = update.message.text
        bot_response = generate_text(user_message)  # Генерация ответа через Hugging Face
        await update.message.reply_text(bot_response)
    except Exception as e:
        await update.message.reply_text(f"Произошла ошибка: {str(e)}")

In [ ]:
def generate_text(prompt):
    return generator(
        prompt,
        max_length=100,  # Максимальная длина ответа
        num_return_sequences=1,  # Количество вариантов ответа
        no_repeat_ngram_size=2,  # Избегание повторяющихся фраз
        top_k=50,  # Ограничение на выбор токенов
        top_p=0.95,  # Фильтрация по вероятности
        temperature=0.7  # Контроль случайности
    )[0]["generated_text"]

In [ ]:
import logging

# Настройка логирования
logging.basicConfig(
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s", level=logging.INFO
)
logger = logging.getLogger(__name__)

# Логирование в обработчиках
async def start(update: Update, context: ContextTypes.DEFAULT_TYPE):
    logger.info(f"Пользователь {update.message.from_user.username} запустил бота.")
    await update.message.reply_text("Привет! Я ваш Нейрокуратор с Hugging Face Transformers. Задайте свой вопрос.")

async def handle_message(update: Update, context: ContextTypes.DEFAULT_TYPE):
    try:
        user_message = update.message.text
        logger.info(f"Пользователь {update.message.from_user.username} написал: {user_message}")
        bot_response = generate_text(user_message)
        await update.message.reply_text(bot_response)
    except Exception as e:
        logger.error(f"Ошибка: {str(e)}")
        await update.message.reply_text(f"Произошла ошибка: {str(e)}")

In [ ]:
async def help_command(update: Update, context: ContextTypes.DEFAULT_TYPE):
    help_text = """
    Я ваш Нейрокуратор! Вот что я умею:
    - Отвечать на ваши вопросы.
    - Генерировать текст по вашему запросу.
    Просто напишите мне что-нибудь, и я постараюсь помочь!
    """
    await update.message.reply_text(help_text)

# Регистрация обработчика команды /help
application.add_handler(CommandHandler("help", help_command))

NameError: name 'application' is not defined

In [ ]:
# Установка библиотек
!pip install python-telegram-bot transformers torch nest_asyncio

# Применение nest_asyncio для работы в Colab
import nest_asyncio
nest_asyncio.apply()

# Импорт библиотек
import torch
from transformers import pipeline
from telegram import Update
from telegram.ext import Application, CommandHandler, MessageHandler, filters, ContextTypes
import logging

# Настройка логирования
logging.basicConfig(
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s", level=logging.INFO
)
logger = logging.getLogger(__name__)

# Токен вашего Telegram-бота
TELEGRAM_TOKEN = "7954916275:AAEoTnpSPG_DRajIGC859AqOmVejyrn7VL0"

# Инициализация модели Hugging Face Transformers
generator = pipeline("text-generation", model="gpt2")  # Используем модель GPT-2

# Функция для генерации текста
def generate_text(prompt):
    return generator(
        prompt,
        max_length=100,
        num_return_sequences=1,
        no_repeat_ngram_size=2,
        top_k=50,
        top_p=0.95,
        temperature=0.7
    )[0]["generated_text"]

# Обработчик команды /start
async def start(update: Update, context: ContextTypes.DEFAULT_TYPE):
    logger.info(f"Пользователь {update.message.from_user.username} запустил бота.")
    await update.message.reply_text("Привет! Я ваш Нейрокуратор с Hugging Face Transformers. Задайте свой вопрос.")

# Обработчик команды /help
async def help_command(update: Update, context: ContextTypes.DEFAULT_TYPE):
    help_text = """
    Я ваш Нейрокуратор! Вот что я умею:
    - Отвечать на ваши вопросы.
    - Генерировать текст по вашему запросу.
    Просто напишите мне что-нибудь, и я постараюсь помочь!
    """
    await update.message.reply_text(help_text)

# Обработчик текстовых сообщений
async def handle_message(update: Update, context: ContextTypes.DEFAULT_TYPE):
    try:
        user_message = update.message.text
        logger.info(f"Пользователь {update.message.from_user.username} написал: {user_message}")
        bot_response = generate_text(user_message)
        await update.message.reply_text(bot_response)
    except Exception as e:
        logger.error(f"Ошибка: {str(e)}")
        await update.message.reply_text(f"Произошла ошибка: {str(e)}")

# Основная функция
def main():
    # Создание приложения бота
    application = Application.builder().token(TELEGRAM_TOKEN).build()

    # Регистрация обработчиков
    application.add_handler(CommandHandler("start", start))
    application.add_handler(CommandHandler("help", help_command))
    application.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, handle_message))

    # Запуск бота
    application.run_polling()

if __name__ == "__main__":
    main()

Device set to use cpu
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


RuntimeError: Cannot close a running event loop

In [ ]:
# Установка модели GPT-Neo
!pip install transformers

# Инициализация модели GPT-Neo
generator = pipeline("text-generation", model="EleutherAI/gpt-neo-2.7B")

config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/10.7G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

Device set to use cpu
